In [96]:
from collections import defaultdict
import unicodedata

In [97]:
def process_text(text):
    preprocessed = []
    for w in text:
        stripped = w.strip()
        if stripped:
            preprocessed.append(stripped)
    titles, contents = [], []
    for i in range(len(preprocessed)//2):
        titles.append(preprocessed[2*i])
        contents.append(preprocessed[2*i+1])
    return titles, contents

In [98]:
def find_closed_marks(text, start, stop, max_len):
    if len(text[start]) > 6 and text[start][:3] == "'''" and text[stop][-3:] == "'''":
        return start, stop
    else:
        while start < min(len(text) - 1, max_len):
            if len(text[start]) > 3 and text[start][:3] == "'''":
                break
            start += 1
        while stop < min(len(text) - 1, max_len):
            if len(text[stop]) > 3 and text[stop][-3:] == "'''":
                break
            stop += 1
        if len(text[start]) > 3 and text[start][:3] == "'''" and len(text[stop]) > 3 and text[stop][-3:] == "'''":
            return start, stop
    return None

def strip_unwanted_punctuation(text):
    unwanted = [",", ")", "("]
    stripped = []
    for word in text:
        if word:
            start = 0
            while start < len(word) and word[start] in unwanted:
                start += 1
            stop = len(word) - 1
            while stop > 1 and word[stop] in unwanted:
                stop -= 1
            if stop - start > 0:
                stripped.append(word[start:(stop+1)])
    return stripped

def strip_all_punctuation(text):
    stripped = []
    for word in text:
        if word:
            start = 0
            while start < len(word) and unicodedata.category(word[start])[0] == 'P' :
                start += 1
            stop = len(word) - 1
            while stop > 1 and unicodedata.category(word[stop])[0] == 'P':
                stop -= 1
            if stop - start > 0:
                stripped.append(word[start:(stop+1)])
    return stripped

In [99]:
def find_synonyms_1(text):
    results = []
    for idx in range(1, min(len(text), 10)):
        if text[idx] in ["lub", "synonimy", "właściwie", "potocznie"] and len(text) > idx + 1:
            res = find_closed_marks(text, idx+1, idx+1, 10)
            if res is not None:
                results.append(text[res[0]:res[1]+1])
    return results

def find_synonyms_2(text):
    results = []
    for idx in range(1, min(len(text), 10)):
        if text[idx][-1] == "," and len(text) > idx + 1:
            res = find_closed_marks(text, idx+1, idx+1, 10)
            if res is not None:
                results.append(text[res[0]:res[1]+1])
    return results

In [100]:
with open("../data/poczatki_wikipediowe.txt", "r") as f:
    text = f.readlines()

titles, contents = process_text(text)

In [101]:
synonyms = defaultdict(list)
for title, content in zip(titles, contents):
    clean_content = strip_unwanted_punctuation(content.split(" "))
    synonyms_1 = find_synonyms_1(clean_content)
    synonyms_2 = find_synonyms_2(clean_content)
    synonyms[title].extend(synonyms_1)
    synonyms[title].extend(synonyms_2)

In [108]:
with open("../other/output_1.txt", "w") as f:
    for k, v in synonyms.items():
        if v:
            v = strip_all_punctuation([" ".join(s) for s in v])
            filtered = list(set([k[4:]] + v))
            if len(filtered) > 1:
                f.write(" # ".join(filtered))
                f.write('\n')